In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
plt.style.use('seaborn-pastel')
from datetime import datetime

In [2]:
# Define start and end date for the financial data
today = datetime.today().strftime('%Y-%m-%d')
startDate = '2017-1-1'

In [3]:
# Pull financial data from yahoo finance for each company ticker and populate dataframe equityPrices.
equities = ['ATVI','AMD','KO','LULU','POR']
equityPrices = pd.DataFrame()

for e in equities:
    equityPrices[e] = wb.DataReader(e,data_source='yahoo',start=startDate,end=today)['Adj Close']

In [4]:
equityPrices.tail()

,ATVI,AMD,KO,LULU,POR
Date,,,,,
2020-05-08,73.139999,53.189999,46.110001,237.990005,45.310001
2020-05-11,75.459999,55.740002,45.540001,245.619995,45.009998
2020-05-12,72.250000,53.759998,44.820000,242.600006,42.849998
2020-05-13,73.669998,52.180000,43.939999,229.389999,41.430000
2020-05-14,71.970001,53.055000,43.805000,233.470001,40.619999


# Visualize Stock Performance

In [ ]:
# Visualize normalized equity prices
normalizedPrice = ((equityPrices/equityPrices.iloc[0]) *100)

In [ ]:
# Define graph elements and create graph of stock prices, normalized to $100.
normalizedPrice.plot(figsize=(15,6))
plt.title('Normalized Equity Prices', fontsize = 24)
plt.xlabel('Date',fontsize = 18)
plt.ylabel('Performance',fontsize = 18)
plt.legend(loc='upper left')
# Plot the graph with all of the defined features above
plt.show()

In [ ]:
# Visualize equity performance over time
equityPrices.plot(figsize=(15,6))
plt.title("Adj. Close Equity Price History",fontsize=24)
plt.xlabel('Date', size=18)
plt.ylabel('Adj. Close Price USD ($)',fontsize=18)
plt.legend(loc='upper left')
plt.show()

# Create and Rank Portfolios

In [ ]:
# Define dailyReturns (log returns) and standardDeviation for each equity. rf is risk free rate used to find annual excessReturns
rf = 0.01
dailyReturns = np.log(1 + equityPrices.pct_change())
excessReturns = dailyReturns.mean()*252 - rf
standardDeviation = dailyReturns.std() * np.sqrt(252)

In [ ]:
# Build covariance matrix that maps the relationship between two equities.  The diagonal left to right is each equity's variance.
covMatrix = dailyReturns.cov() * 252

In [ ]:
numEquities = len(equities)

In [ ]:
# Create possible portfolios and store each portfolio's return and risk (standard deviation) in two lists
portfolioReturns = []
portfolioRisk = []

In [ ]:
# weightsDictonary stores the weights that are generated to create portfolios.  WeightsList is a list of the key names of weightDictonary
weightsDictonary = {}
for e in equities:
    weightsDictonary[str(e) + '_weight'] = []
weightsList = list(weightsDictonary)

In [ ]:
# Append the return and risk for portfolio with randomly generated weights assigned to each equity.
for x in range(1000):
    weights = np.random.random(numEquities)
    weights /= np.sum(weights)
    portfolioReturns.append(np.sum(weights * excessReturns))
    portfolioRisk.append(np.sqrt(np.dot(weights.T,np.dot(covMatrix,weights))))
# For reach iteration store the randomly generated weights as values in weightsDictionary    
    for w in range(len(weights)):
        weightsDictonary[weightsList[w]].append(weights[w])

In [ ]:
# Change into to numpy array
portfolioReturns = np.array(portfolioReturns)
portfolioRisk = np.array(portfolioRisk)

for i in range(len(weightsDictonary)):
    weightsDictonary[weightsList[i]] = np.array(weightsDictonary[weightsList[i]])

In [ ]:
# Populate dataframe with Return and Risk for each portfolio along with its respective weights for each equity.
portfolios = pd.DataFrame({**{'Risk_Premium':portfolioReturns,'Standard_Deviation':portfolioRisk},**weightsDictonary})
portfolios.eval('Sharpe_Ratio=Risk_Premium/Standard_Deviation',inplace = True)
firstColumn = portfolios.pop('Sharpe_Ratio')
portfolios.insert(0,'Sharpe_Ratio',firstColumn)

In [ ]:
# Sort portfolios by Sharpe_Ratio descending. Puts the optimal portfolio in the set at the top of data frame
portfolios.sort_values(by='Sharpe_Ratio',ascending=False)

In [ ]:
# Assign index of optimal portfolio to variable to plot on a graph.
optimalPortfolio = int(portfolios.idxmax('index')['Sharpe_Ratio'])

In [ ]:
# Plot all of the portfolios to visualize the efficient frontier. Mark the optimal portfolio with a red x along efficient frontier 
plt.figure(figsize=(20,8))
optimalX = portfolios.iloc[optimalPortfolio].loc['Standard_Deviation']
optimalY = portfolios.iloc[optimalPortfolio].loc['Risk_Premium']
portfolioScatter = plt.scatter(portfolioRisk,portfolioReturns)
optimalPoint = plt.scatter(optimalX,optimalY,s=100,marker='x',color='red')
plt.title('Efficient Frontier',fontsize = 24)
plt.xlabel('Standard Deviation',fontsize = 18)
plt.ylabel('Return',fontsize=18)
plt.legend((portfolioScatter,optimalPoint),('Portfolios','Optimal Portfolio'),loc='upper left')
plt.show()